In [2]:
print("Hello")

Hello


# install libraries

In [5]:
# pip install apify-client
# !pip install textblob
# pip install Praw

   ---------------------------------------- 0.0/624.3 kB ? eta -:--:--
   ---------------------------------------- 624.3/624.3 kB 5.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 1.5/1.5 MB 8.0 MB/s eta 0:00:00



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


# Extract data from reddddit

In [6]:
import praw

reddit = praw.Reddit(
    client_id="h7SCaVUWMMskrKxda6dW7K6aA",
    client_secret="ndAIj2lKITQBJ6NzHAFD10FkyuSPZer2A",
    user_agent="my_app"
)

# Example: fetch top posts from r/electronic_cigarette
subreddit = reddit.subreddit("maker")

for post in subreddit.top(limit=5):
    print({
        "datetime": post.created_utc,
        "niche": "vape",
        "product": None,
        "hashtags": None,
        "description": post.title,
        "country": None,
        "area": None,
        "store/channel_name": post.author.name if post.author else None,
        "social_platform": "reddit"
    })


{'datetime': 1754839769.0, 'niche': 'vape', 'product': None, 'hashtags': None, 'description': 'I built a machine that turns thread into portraits', 'country': None, 'area': None, 'store/channel_name': 'Quiet_Compote_6803', 'social_platform': 'reddit'}
{'datetime': 1462377072.0, 'niche': 'vape', 'product': None, 'hashtags': None, 'description': 'Maker AMA Series: I am Adam Savage, maker, editor-in-chief of Tested.com and former host of MythBusters. AMAAM! (Ask Me Anything About Making!)', 'country': None, 'area': None, 'store/channel_name': 'mistersavage', 'social_platform': 'reddit'}
{'datetime': 1742483895.0, 'niche': 'vape', 'product': None, 'hashtags': None, 'description': 'I made a really cool lamp and I’m really proud of it!', 'country': None, 'area': None, 'store/channel_name': 'careyi4', 'social_platform': 'reddit'}
{'datetime': 1740090113.0, 'niche': 'vape', 'product': None, 'hashtags': None, 'description': 'I made an eternal flame in a tiny dumpster that burns until the end of